In [1]:
from pymongo import MongoClient
import pandas as pd
from bson.objectid import ObjectId  # For handling MongoDB ObjectId
import os
import getpass

In [2]:
client = MongoClient(os.environ["ECOMMERCE__CONNECTION_STRING"])
db = client["e-commerce_cleaned"]
serverStatusResult = db.command("serverStatus")

/toolkit-cache/0.2.0/python3.9/kernel-libs/lib/python3.9/site-packages/pymongo/ocsp_support.py:284: CryptographyDeprecationWarning: Properties that return a naïve datetime object have been deprecated. Please switch to next_update_utc.
  if response.next_update and response.next_update < now:
/toolkit-cache/0.2.0/python3.9/kernel-libs/lib/python3.9/site-packages/pymongo/ocsp_support.py:280: CryptographyDeprecationWarning: Properties that return a naïve datetime object have been deprecated. Please switch to this_update_utc.
  if response.this_update > now:
/toolkit-cache/0.2.0/python3.9/kernel-libs/lib/python3.9/site-packages/pymongo/ocsp_cache.py:65: CryptographyDeprecationWarning: Properties that return a naïve datetime object have been deprecated. Please switch to next_update_utc.
  if value.next_update is None:
/toolkit-cache/0.2.0/python3.9/kernel-libs/lib/python3.9/site-packages/pymongo/ocsp_cache.py:71: CryptographyDeprecationWarning: Properties that return a naïve datetime object

In [3]:
# Function to show summary and statistics
def view_collection(collection_name):
    collection = db[collection_name]
    data = list(collection.find({}, {"_id": 0}))  # Exclude `_id` for simplicity
    if not data:
        print("No data available in this collection.")
        return
    df = pd.DataFrame(data)
    print("Head of the dataset:\n", df.head())
    print("\nTail of the dataset:\n", df.tail())
    print("\nSummary statistics:\n", df.describe(include="all"))


# Function to insert a new record
def insert_into_collection(collection_name):
    collection = db[collection_name]
    example_doc = collection.find_one({}, {"_id": 0})
    if not example_doc:
        print("No existing documents to infer fields.")
        return
    print("Provide values for the following fields (leave blank to omit):")
    new_doc = {}
    for field in example_doc.keys():
        value = input(f"{field}: ")
        if value:
            new_doc[field] = value
    collection.insert_one(new_doc)
    print("New document inserted successfully.")


# Function to update an existing record
def update_collection(collection_name):
    collection = db[collection_name]
    record_id = input("Enter the unique identifier (_id) of the record to update: ")
    try:
        record_id = ObjectId(record_id)  # Convert string to ObjectId
    except Exception:
        print("Invalid _id format.")
        return
    field_to_update = input("Enter the field to update: ")
    new_value = input("Enter the new value: ")
    result = collection.update_one({"_id": record_id}, {"$set": {field_to_update: new_value}})
    if result.matched_count > 0:
        print("Record updated successfully.")
    else:
        print("No matching record found.")


# Function to delete a record
def delete_from_collection(collection_name):
    collection = db[collection_name]
    record_id = input("Enter the unique identifier (_id) of the record to delete: ")
    try:
        record_id = ObjectId(record_id)  # Convert string to ObjectId
    except Exception:
        print("Invalid _id format.")
        return
    result = collection.delete_one({"_id": record_id})
    if result.deleted_count > 0:
        print("Record deleted successfully.")
    else:
        print("No matching record found.")




In [4]:

USERS = {
    "admin": {"password": "admin123", "role": "admin"},
    "user": {"password": "user123", "role": "user"}
}

# Function to authenticate the user
def authenticate():
    username = input("Enter username: ")
    password = getpass.getpass("Enter password: ")

    if username in USERS and USERS[username]["password"] == password:
        print(f"Welcome, {username}!")
        return USERS[username]["role"]
    else:
        print("Invalid username or password.")
        return None



# Main program
def main():
    role = authenticate()
    if not role:
        return

    while True:
        collections = db.list_collection_names()  
        print("\nAvailable collections:")
        for idx, name in enumerate(collections, start=1):
            print(f"{idx}. {name}")
        print("0. Exit")

        try:
            collection_choice = int(input("\nSelect a collection by number (or 0 to exit): ")) - 1
            if collection_choice == -1:  # User selects 0 (exit)
                print("Exiting the program.")
                break
            if collection_choice < 0 or collection_choice >= len(collections):
                print("Invalid choice. Please try again.")
                continue

            selected_collection = collections[collection_choice]
            print(f"\nYou selected: {selected_collection}")

            while True:
                print("\nAvailable actions:")
                if role == "admin":
                    print("1. View collection")
                    print("2. Insert into collection")
                    print("3. Update collection")
                    print("4. Delete from collection")
                    print("0. Back to collection selection")
                else:  # Regular user
                    print("1. View collection")
                    print("0. Back to collection selection")

                action_choice = int(input("Select an action by number: "))
                if action_choice == 0:  # Go back to collection selection
                    break
                if role == "admin":
                    if action_choice == 1:
                        view_collection(selected_collection)
                    elif action_choice == 2:
                        insert_into_collection(selected_collection)
                    elif action_choice == 3:
                        update_collection(selected_collection)
                    elif action_choice == 4:
                        delete_from_collection(selected_collection)
                    else:
                        print("Invalid action. Please try again.")
                else:  # Regular user can only view
                    if action_choice == 1:
                        view_collection(selected_collection)
                    else:
                        print("Invalid action. Please try again.")
        except ValueError:
            print("Invalid input. Please enter a number.")

if __name__ == "__main__":
    main()


Welcome, admin!

Available collections:
1. Amazon-Sale-Report-Cleaned
2. Sales-Report-Cleaned
3. International-Sale-Report-Cleaned
4. Sales-March-2021-Cleaned
5. Sales-May-2022-Cleaned
0. Exit

You selected: Sales-Report-Cleaned

Available actions:
1. View collection
2. Insert into collection
3. Update collection
4. Delete from collection
0. Back to collection selection
Provide values for the following fields (leave blank to omit):
New document inserted successfully.

Available actions:
1. View collection
2. Insert into collection
3. Update collection
4. Delete from collection
0. Back to collection selection
Provide values for the following fields (leave blank to omit):
New document inserted successfully.

Available actions:
1. View collection
2. Insert into collection
3. Update collection
4. Delete from collection
0. Back to collection selection
Invalid input. Please enter a number.

Available collections:
1. Amazon-Sale-Report-Cleaned
2. Sales-Report-Cleaned
3. International-Sale-Rep

KeyboardInterrupt: Interrupted by user

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=e322a799-a681-4754-b873-85c47d774f17' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>